## Measuring regional outdoor recreation in the Bay Area using social media, in 3 different categories of land:  
1. Priority Conservation Areas (PCAs),
2. Protected Areas (Bay Area Protected Areas Database - BPAD) 
3. Natural landcover types (NLCD)

And summarizing how much of the recreation in each category is exposed to sea-level rise, based on 10 different SLR scenarios.

This workflow depends on results from:
    * slr_rasters_align.py
    * userdays_rasteraoi.r
    * pud_rasterize.py

In [1]:
import numpy as np
from osgeo import gdal
gdal.UseExceptions()
import pygeoprocessing.geoprocessing as pgp
import os
import glob
import pandas as pd

In [2]:
workspace = '../data/bcdc_slr_regional_rec'
# base_uri = os.path.join(workspace, 'base_grid.tif')

In [3]:
nlcd_uri = '../data/bcdc_othernaturalareas/NaturalAreas_ForDave/nlcd_nodevt_utm.tif'
nlcd_src = gdal.Open(nlcd_uri)

In [4]:
def raster2array(raster_uri):
    '''
    helper function to go straight from raster filename to numpy array
    '''
    src = gdal.Open(raster_uri)
    band1 = src.GetRasterBand(1)
    rows = src.RasterYSize
    cols = src.RasterXSize
    arr = band1.ReadAsArray(0, 0, cols, rows)
    return(arr)

In [5]:
pud_uri = '../data/flickr/nlcd_grid_pud/pud_nlcdgrid.tif'
pud = raster2array(pud_uri)

In [ ]:
# pgp.new_raster_from_base(nlcd_uri, base_uri, gdal.GDT_Int16, [99], [0])

## Load AOIs and burn them onto a standard base raster

In [6]:
# BPAD
bpad_uri = os.path.join(workspace, 'bpad.tif')
pgp.new_raster_from_base(nlcd_uri, bpad_uri, gdal.GDT_Int16, [99], [0])
shp_uri = '../data/bcdc_othernaturalareas/NaturalAreas_ForDave/BayAreaProtectedLands_utm.shp'
pgp.rasterize(shp_uri, bpad_uri, [1], None)

In [7]:
# PCA
pca_uri = os.path.join(workspace, 'pca.tif')
pgp.new_raster_from_base(nlcd_uri, pca_uri, gdal.GDT_Int16, [99], [0])
shp_uri = '../data/pca/shapefiles/Priority_Conservation_Areas_current.shp'
pgp.rasterize(shp_uri, pca_uri, [1], None)

## then read AOIs as 2d arrays for querying

In [8]:
# Natural Lands - make 2d array 0s and 1s where 1s are all lulc classes other than development
# in the original datasource, development lulc's have already been assigned nodata
band1 = nlcd_src.GetRasterBand(1)
nodata = band1.GetNoDataValue()
rows = nlcd_src.RasterYSize
cols = nlcd_src.RasterXSize
vals = band1.ReadAsArray(0, 0, cols, rows)
nlcd = np.ones_like(vals)
nlcd[vals == nodata] = 0

In [9]:
bpad = raster2array(bpad_uri)
pca = raster2array(pca_uri)

In [10]:
nlcd.shape == bpad.shape == pca.shape == pud.shape

True

### testing some logic on small arrays...

In [17]:
x = np.zeros((3,3))
x[:2,:2] = 1
y = np.zeros((3,3))
y[:1,:2] = 1
w = np.zeros((3,3))
w[0,1] = 1
z = np.flip(np.arange(9).reshape(3, 3), 0)

In [18]:
x

array([[1., 1., 0.],
       [1., 1., 0.],
       [0., 0., 0.]])

In [19]:
y

array([[1., 1., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [20]:
w

array([[0., 1., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [21]:
z

array([[6, 7, 8],
       [3, 4, 5],
       [0, 1, 2]])

In [22]:
z[(x != 0) & (y != 0) & (w != 1)]

array([6])

## Count the photo-user-days across all grid cells included in combinations of the AOIs and SLR scenarios. 

#### All calculations below exclude developed areas (based on lulc development classes)

In [23]:
slr_rasters = glob.glob('../data/pca/bcdc_slr/raster/aligned/Inundate*.tif')

In [24]:
# In PCA network, by SLR scenario
pca_slr = {}

for slr_uri in slr_rasters:
    print(slr_uri)
    slr = raster2array(slr_uri)
    pud.shape == nlcd.shape == pca.shape == slr.shape
    puds = np.sum(pud[(nlcd != 0) & \
                      (pca != 0) & \
                      (slr != 0)])
    pca_slr[os.path.basename(slr_uri)] = puds
    
puds_slr0 = np.sum(pud[(nlcd != 0) & \
                      (pca != 0)])
pca_slr['Inundate_0'] = puds_slr0

../data/pca/bcdc_slr/raster/aligned/Inundate_66.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_77.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_12.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_52.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_108.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_96.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_48.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_24.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_36.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_84.tif


In [25]:
# In BPAD network and not in PCA network, by SLR scenario
bpad_slr = {}

for slr_uri in slr_rasters:
    print(slr_uri)
    slr = raster2array(slr_uri)
    puds = np.sum(pud[(nlcd != 0) & \
                      (bpad != 0) & \
                      (slr != 0) & \
                      (pca != 1)])
    bpad_slr[os.path.basename(slr_uri)] = puds
        
puds_slr0 = np.sum(pud[(nlcd != 0) & \
                      (bpad != 0) & \
                      (pca != 1)])
bpad_slr['Inundate_0'] = puds_slr0

../data/pca/bcdc_slr/raster/aligned/Inundate_66.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_77.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_12.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_52.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_108.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_96.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_48.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_24.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_36.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_84.tif


In [26]:
# In Natural Lands and not in BPAD and PCA networks, by SLR scenario
nlcd_slr = {}

for slr_uri in slr_rasters:
    print(slr_uri)
    slr = raster2array(slr_uri)
    puds = np.sum(pud[(nlcd != 0) & \
                      (slr != 0) & \
                      (pca != 1) & \
                      (bpad != 1)])
    
    nlcd_slr[os.path.basename(slr_uri)] = puds
    
puds_slr0 = np.sum(pud[(nlcd != 0) & \
                      (pca != 1) & \
                      (bpad != 1)])
nlcd_slr['Inundate_0'] = puds_slr0

../data/pca/bcdc_slr/raster/aligned/Inundate_66.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_77.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_12.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_52.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_108.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_96.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_48.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_24.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_36.tif
../data/pca/bcdc_slr/raster/aligned/Inundate_84.tif


## Compile results

In [27]:
pca_df = pd.DataFrame(pca_slr.items(), columns=['scenario', 'pca'])
bpad_df = pd.DataFrame(bpad_slr.items(), columns=['scenario', 'bpad'])
nlcd_df = pd.DataFrame(nlcd_slr.items(), columns=['scenario', 'nlcd'])

In [28]:
results = pd.merge(pd.merge(pca_df, bpad_df, on='scenario'), nlcd_df, on='scenario')

In [29]:
results['slr_inches'] = pd.to_numeric(results['scenario'].str.extract('([0-9]?[0-9]?[0-9])'))

/home/dmf/miniconda3/envs/geowork2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  if __name__ == '__main__':


In [30]:
results.sort_values(by='slr_inches')

,scenario,pca,bpad,nlcd,slr_inches
2,Inundate_0,120622,245852,140413,0
10,Inundate_12.tif,1332,3364,4274,12
7,Inundate_24.tif,3079,4381,5615,24
0,Inundate_36.tif,3469,6871,6974,36
1,Inundate_48.tif,4945,8189,9022,48
9,Inundate_52.tif,5037,8736,10383,52
4,Inundate_66.tif,5396,16157,14624,66
5,Inundate_77.tif,7767,17554,20666,77
6,Inundate_84.tif,7883,18493,21556,84
3,Inundate_96.tif,8012,19743,23717,96


In [31]:
results.to_csv(os.path.join(workspace, 'results.csv'))